<a href="https://colab.research.google.com/github/royn5618/Medium_Blog_Codes/blob/master/Emotion%20Detection/EmotionClassifier_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt

# to avoid warnings 
import warnings
warnings.filterwarnings("ignore")

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Keras imports
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, LSTM

# Scikit Learn imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
STOPWORDS = stopwords.words('english')
PORTER_STEMMER = PorterStemmer()


In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

In [ ]:
train_data = pd.read_csv('train.txt', sep=';', names=['text', 'emotion'])
train_data.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
test_data = pd.read_csv('test.txt', sep=';', names=['text', 'emotion'])
test_data.head()

,text,emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [ ]:
val_data = pd.read_csv('val.txt', sep=';', names=['text', 'emotion'])
val_data.head()

,text,emotion
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [ ]:
def preprocess_text(text):
    filtered_text = []
    for each_word in word_tokenize(text):
        if each_word not in STOPWORDS:
            filtered_text.append(PORTER_STEMMER.stem(each_word))
    return " ".join(filtered_text)

In [ ]:
train_data['text'] = train_data.text.apply(preprocess_text)
test_data['text'] = test_data.text.apply(preprocess_text)
val_data['text'] = val_data.text.apply(preprocess_text)

In [ ]:
train_data["emotion"] = train_data["emotion"].astype('category')
train_data["emotion_label"] = train_data["emotion"].cat.codes
train_data.head()

,text,emotion,emotion_label
0,didnt feel humili,sadness,4
1,go feel hopeless damn hope around someon care ...,sadness,4
2,im grab minut post feel greedi wrong,anger,0
3,ever feel nostalg fireplac know still properti,love,3
4,feel grouchi,anger,0


In [ ]:
test_data["emotion"] = test_data["emotion"].astype('category')
test_data["emotion_label"] = test_data["emotion"].cat.codes
test_data.head()

val_data["emotion"] = val_data["emotion"].astype('category')
val_data["emotion_label"] = val_data["emotion"].cat.codes
val_data.head()

train_features, train_labels = train_data['text'], tf.one_hot(
    train_data["emotion_label"], 6)
test_features, test_labels = test_data['text'], tf.one_hot(
    test_data["emotion_label"], 6)
val_features, val_labels = val_data['text'], tf.one_hot(
    val_data["emotion_label"], 6)

def get_labels_from_oh_code(oh_code):
    """ Takes in one-hot encoded matrix
    Returns a list of decoded categories"""
    label_code = np.argmax(oh_code, axis=1)
#     print(label_code)
    label = test_data.emotion.cat.categories[label_code]
#     print(list(label))
    return list(label)

In [ ]:
total_data = ' '.join(list(train_data.text))
len(set(word_tokenize(total_data))) # vocab size

10375

In [ ]:
vocab_size = 10000
vector_size = 300
max_seq_len = 20

tokenizer = Tokenizer(oov_token = "<OOV>", num_words=vocab_size, lower=True)
tokenizer.fit_on_texts(train_data['text'])

sequences_train = tokenizer.texts_to_sequences(train_data['text'])
sequences_test = tokenizer.texts_to_sequences(test_data['text'])
sequences_val = tokenizer.texts_to_sequences(val_data['text'])

padded_train = pad_sequences(sequences_train, padding = 'post', maxlen=max_seq_len)
padded_test = pad_sequences(sequences_test, padding = 'post', maxlen=max_seq_len)
padded_val = pad_sequences(sequences_val, padding = 'post', maxlen=max_seq_len)

In [ ]:
def model_builder(hp):
    model = Sequential()
    hp_vector_size = hp.Int('vector_size', min_value=100, max_value=500, step=100)
    model.add(
        Embedding(input_dim=vocab_size,
                  output_dim=vector_size,
                  input_length=max_seq_len))
    hp_dropout_rate = hp.Float('dropout_rate', min_value=0.6, max_value=0.9, step=0.1)
    model.add(Dropout(hp_dropout_rate))
    hp_lstm_units1 = hp.Int('lstm_units1', min_value=32, max_value=512, step=32)
    model.add(LSTM(hp_lstm_units1,return_sequences=True))
    hp_lstm_units2 = hp.Int('lstm_units2', min_value=16, max_value=512, step=32)
    model.add(LSTM(hp_lstm_units2))
    model.add(Dense(6,activation='softmax'))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
              loss='categorical_crossentropy',
              metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
    return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective=kt.Objective("val_precision", direction="max"),
                     max_epochs=20,
                     factor=3)

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_precision', patience=5)

In [ ]:
tuner.search(padded_train, train_labels, 
             epochs=20, 
             validation_data=(padded_val, val_labels), 
             callbacks=[stop_early],
             directory='ModelTunerCheckPoints',
             project_name='EmotionClassifierTuner'
             )

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 30 Complete [00h 00m 41s]
val_precision: 0.8796389102935791

Best val_precision So Far: 0.9182839393615723
Total elapsed time: 00h 17m 43s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps.get('vector_size')

200

In [ ]:
best_hps.get('dropout_rate')

0.7999999999999999

In [ ]:
best_hps.get('lstm_units1')

480

In [ ]:
best_hps.get('lstm_units2')

272

In [ ]:
best_hps.get('learning_rate')

0.001

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model_best_hp = tuner.hypermodel.build(best_hps)
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss',
                                  mode='min',
                                  patience=5,
                                  verbose=1,
                                  restore_best_weights=True)  
]
history = model_best_hp.fit(padded_train, 
                            train_labels, 
                            epochs=20,
                            callbacks=callbacks,
                            validation_data=(padded_val, val_labels))

Epoch 1/20
500/500 [==============================] - 9s 13ms/step - loss: 1.3170 - recall_4: 0.2035 - precision_4: 0.6635 - val_loss: 0.7293 - val_recall_4: 0.6300 - val_precision_4: 0.8618
Epoch 2/20
500/500 [==============================] - 6s 11ms/step - loss: 0.5564 - recall_4: 0.7654 - precision_4: 0.8574 - val_loss: 0.3351 - val_recall_4: 0.8725 - val_precision_4: 0.9141
Epoch 3/20
500/500 [==============================] - 6s 11ms/step - loss: 0.3260 - recall_4: 0.8742 - precision_4: 0.9054 - val_loss: 0.2914 - val_recall_4: 0.8960 - val_precision_4: 0.9148
Epoch 4/20
500/500 [==============================] - 6s 11ms/step - loss: 0.2586 - recall_4: 0.8990 - precision_4: 0.9190 - val_loss: 0.2598 - val_recall_4: 0.8935 - val_precision_4: 0.9141
Epoch 5/20
500/500 [==============================] - 6s 11ms/step - loss: 0.2089 - recall_4: 0.9137 - precision_4: 0.9322 - val_loss: 0.2680 - val_recall_4: 0.8975 - val_precision_4: 0.9153
Epoch 6/20
500/500 [=========================

In [ ]:
# Model Evaluation on Train Data
y_pred_one_hot_encoded = (model_best_hp.predict(padded_train)> 0.5).astype("int32")
y_pred = np.array(tf.argmax(y_pred_one_hot_encoded, axis=1))
print(classification_report(train_data['emotion_label'], y_pred))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      2159
           1       0.95      0.90      0.92      1937
           2       0.96      0.99      0.97      5362
           3       0.98      0.84      0.90      1304
           4       0.98      0.98      0.98      4666
           5       0.92      0.80      0.86       572

    accuracy                           0.96     16000
   macro avg       0.95      0.92      0.93     16000
weighted avg       0.96      0.96      0.96     16000



In [ ]:
# Model Evaaluation on Validation Data
y_pred_one_hot_encoded = (model_best_hp.predict(padded_val)> 0.5).astype("int32")
y_pred = np.array(tf.argmax(y_pred_one_hot_encoded, axis=1))
print(classification_report(val_data['emotion_label'], y_pred))

              precision    recall  f1-score   support

           0       0.85      0.96      0.90       275
           1       0.90      0.81      0.85       212
           2       0.92      0.94      0.93       704
           3       0.88      0.73      0.80       178
           4       0.94      0.96      0.95       550
           5       0.89      0.80      0.84        81

    accuracy                           0.91      2000
   macro avg       0.90      0.87      0.88      2000
weighted avg       0.91      0.91      0.91      2000



In [ ]:
# Model Evaaluation on Test Data
y_pred_one_hot_encoded = (model_best_hp.predict(padded_test)> 0.5).astype("int32")
y_pred = np.array(tf.argmax(y_pred_one_hot_encoded, axis=1))
print(classification_report(test_data['emotion_label'], y_pred))

              precision    recall  f1-score   support

           0       0.81      0.93      0.87       275
           1       0.89      0.88      0.89       224
           2       0.90      0.94      0.92       695
           3       0.81      0.66      0.73       159
           4       0.96      0.92      0.94       581
           5       0.79      0.62      0.69        66

    accuracy                           0.89      2000
   macro avg       0.86      0.83      0.84      2000
weighted avg       0.89      0.89      0.89      2000

